In [2]:
import librosa
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import random
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import torch
import torchmetrics
import os
import warnings
warnings.filterwarnings('ignore')

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

class Config:
    SR = 32000
    N_MFCC = 13
    ROOT_FOLDER = './'
    N_CLASSES = 2
    BATCH_SIZE = 64
    N_EPOCHS = 50
    LR = 1e-4
    SEED = 42
    N_MODELS = 5  # Number of models for ensemble

CONFIG = Config()

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CONFIG.SEED)

df = pd.read_csv('./train.csv')
train, val, _, _ = train_test_split(df, df['label'], test_size=0.2, random_state=CONFIG.SEED)

def get_mfcc_feature(df, train_mode=True):
    features = []
    labels = []
    for _, row in tqdm(df.iterrows()):
        y, sr = librosa.load(row['path'], sr=CONFIG.SR)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=CONFIG.N_MFCC)
        mfcc = np.mean(mfcc.T, axis=0)
        features.append(mfcc)

        if train_mode:
            label = row['label']
            label_vector = np.zeros(CONFIG.N_CLASSES, dtype=float)
            label_vector[0 if label == 'fake' else 1] = 1
            labels.append(label_vector)

    if train_mode:
        return features, labels
    return features

train_mfcc, train_labels = get_mfcc_feature(train, True)
val_mfcc, val_labels = get_mfcc_feature(val, True)

class CustomDataset(Dataset):
    def __init__(self, mfcc, label):
        self.mfcc = mfcc
        self.label = label

    def __len__(self):
        return len(self.mfcc)

    def __getitem__(self, index):
        if self.label is not None:
            return self.mfcc[index], self.label[index]
        return self.mfcc[index]

train_dataset = CustomDataset(train_mfcc, train_labels)
val_dataset = CustomDataset(val_mfcc, val_labels)

train_loader = DataLoader(train_dataset, batch_size=CONFIG.BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=CONFIG.BATCH_SIZE, shuffle=False)

class ImprovedMLP(nn.Module):
    def __init__(self, input_dim=CONFIG.N_MFCC, hidden_dim=256, output_dim=CONFIG.N_CLASSES):
        super(ImprovedMLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.bn2 = nn.BatchNorm1d(hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = self.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = self.fc3(x)
        x = torch.sigmoid(x)
        return x

from sklearn.metrics import roc_auc_score

def train(model, optimizer, train_loader, val_loader, device, scheduler=None):
    model.to(device)
    criterion = nn.BCELoss().to(device)
    
    best_val_score = 0
    best_model = None
    
    for epoch in range(1, CONFIG.N_EPOCHS+1):
        model.train()
        train_loss = []
        for features, labels in tqdm(iter(train_loader)):
            features = features.float().to(device)
            labels = labels.float().to(device)
            
            optimizer.zero_grad()
            
            output = model(features)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
        
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        
        if scheduler:
            scheduler.step(_val_loss)
        
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val AUC : [{_val_score:.5f}]')
            
        if best_val_score < _val_score:
            best_val_score = _val_score
            best_model = model
    
    return best_model, best_val_score

def multiLabel_AUC(y_true, y_scores):
    auc_scores = []
    for i in range(y_true.shape[1]):
        auc = roc_auc_score(y_true[:, i], y_scores[:, i])
        auc_scores.append(auc)
    mean_auc_score = np.mean(auc_scores)
    return mean_auc_score
    
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss, all_labels, all_probs = [], [], []
    
    with torch.no_grad():
        for features, labels in tqdm(iter(val_loader)):
            features = features.float().to(device)
            labels = labels.float().to(device)
            
            probs = model(features)
            
            loss = criterion(probs, labels)

            val_loss.append(loss.item())

            all_labels.append(labels.cpu().numpy())
            all_probs.append(probs.cpu().numpy())
        
        _val_loss = np.mean(val_loss)

        all_labels = np.concatenate(all_labels, axis=0)
        all_probs = np.concatenate(all_probs, axis=0)
        
        auc_score = multiLabel_AUC(all_labels, all_probs)
    
    return _val_loss, auc_score

def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    predictions = []
    with torch.no_grad():
        for features in tqdm(iter(test_loader)):
            features = features.float().to(device)
            probs = model(features)
            probs = probs.cpu().detach().numpy()
            predictions += probs.tolist()
    return predictions

# Training multiple models for ensemble
models = []
val_scores = []
for _ in range(CONFIG.N_MODELS):
    model = ImprovedMLP()
    optimizer = torch.optim.Adam(params=model.parameters(), lr=CONFIG.LR)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5)
    best_model, best_val_score = train(model, optimizer, train_loader, val_loader, device, scheduler=scheduler)
    models.append(best_model)
    val_scores.append(best_val_score)

# Select the best model based on validation score
best_model_index = np.argmax(val_scores)
best_model = models[best_model_index]

test = pd.read_csv('./test.csv')
test_mfcc = get_mfcc_feature(test, False)
test_dataset = CustomDataset(test_mfcc, None)
test_loader = DataLoader(test_dataset, batch_size=CONFIG.BATCH_SIZE, shuffle=False)

# Inference with the best model
preds = inference(best_model, test_loader, device)

submit = pd.read_csv('./sample_submission.csv')
submit.iloc[:, 1:] = preds
submit.head()

submit.to_csv('./best_model_submit.csv', index=False)


44350it [21:22, 34.59it/s]
11088it [05:33, 33.24it/s]
100%|██████████| 174/174 [00:00<00:00, 289.95it/s]


Epoch [1], Train Loss : [0.52655] Val Loss : [0.38059] Val AUC : [0.92087]


100%|██████████| 174/174 [00:00<00:00, 280.76it/s]


Epoch [2], Train Loss : [0.41531] Val Loss : [0.32498] Val AUC : [0.93996]


100%|██████████| 174/174 [00:00<00:00, 297.37it/s]


...

100%|██████████| 174/174 [00:00<00:00, 291.92it/s]


Epoch [49], Train Loss : [0.17744] Val Loss : [0.10676] Val AUC : [0.99328]


100%|██████████| 174/174 [00:00<00:00, 302.52it/s]


Epoch [50], Train Loss : [0.17663] Val Loss : [0.10512] Val AUC : [0.99323]


50000it [31:27, 26.48it/s]
100%|██████████| 782/782 [00:02<00:00, 331.86it/s]


In [1]:
import numpy as np
import pandas as pd
import os
import librosa
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import RandomOverSampler
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import random
import warnings
from sklearn.metrics import roc_auc_score, mean_squared_error
from sklearn.calibration import calibration_curve

warnings.filterwarnings('ignore')

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Using device: {device}")

class Config:
    SR = 32000
    N_MFCC = 13
    N_FEATURES = 193  # 변경된 특징 벡터 길이
    ROOT_FOLDER = './'
    N_CLASSES = 2
    BATCH_SIZE = 32  # 설정된 하이퍼파라미터
    N_EPOCHS = 10
    LR = 0.00037191891209066374  # 설정된 하이퍼파라미터
    SEED = 42
    HIDDEN_DIM = 256  # 설정된 하이퍼파라미터
    DROPOUT_RATE = 0.41872215774632654  # 설정된 하이퍼파라미터

CONFIG = Config()

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CONFIG.SEED)

# 데이터 로드 및 전처리
df = pd.read_csv(os.path.join(CONFIG.ROOT_FOLDER, 'train.csv'))

def label_encoder(column):
    le = LabelEncoder().fit(column)
    print(column.name, le.classes_)
    return le.transform(column)

df['class'] = label_encoder(df['label'])

# 다양한 음성 특징 추출 함수
def extract_features(file_path):
    y, sr = librosa.load(file_path, sr=CONFIG.SR)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=CONFIG.N_MFCC)
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    mel = librosa.feature.melspectrogram(y=y, sr=sr)
    contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
    tonnetz = librosa.feature.tonnetz(y=librosa.effects.harmonic(y), sr=sr)
    
    features = np.concatenate((
        np.mean(mfcc, axis=1), 
        np.std(mfcc, axis=1), 
        np.mean(chroma, axis=1), 
        np.std(chroma, axis=1), 
        np.mean(mel, axis=1), 
        np.std(mel, axis=1), 
        np.mean(contrast, axis=1), 
        np.std(contrast, axis=1), 
        np.mean(tonnetz, axis=1), 
        np.std(tonnetz, axis=1)
    ), axis=0)
    
    return features

def get_features_and_labels(df, train_mode=True):
    features = []
    labels = []
    for _, row in tqdm(df.iterrows()):
        file_path = os.path.join(CONFIG.ROOT_FOLDER, row['path'][2:])  # './train/' 또는 './test/' 제거
        if os.path.isfile(file_path):
            features.append(extract_features(file_path))
            if train_mode:
                labels.append(row['class'])
    if train_mode:
        return np.array(features), np.array(labels)
    return np.array(features)

# Train 데이터에서 특징 추출
X, y = get_features_and_labels(df, True)

# 데이터 불균형 해결
ros = RandomOverSampler(random_state=CONFIG.SEED)
X_resampled, y_resampled = ros.fit_resample(X, y)
y_resampled = torch.tensor(y_resampled).long()  # 정수형으로 변환
y_resampled = torch.nn.functional.one_hot(y_resampled, num_classes=CONFIG.N_CLASSES).float()

# 데이터 분할
X_train, X_val, y_train, y_val = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=CONFIG.SEED)

# PyTorch Dataset 클래스 정의
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, index):
        return self.features[index], self.labels[index]

train_dataset = CustomDataset(X_train, y_train)
val_dataset = CustomDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=CONFIG.BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=CONFIG.BATCH_SIZE, shuffle=False)

# CNN+RNN 모델 정의
class CNNRNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout_rate):
        super(CNNRNN, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(1, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Conv1d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
        )
        self.rnn = nn.LSTM(input_size=128, hidden_size=hidden_dim, num_layers=2, batch_first=True, bidirectional=True, dropout=dropout_rate)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = x.unsqueeze(1)  # Add channel dimension
        x = self.cnn(x)
        x, _ = self.rnn(x.permute(0, 2, 1))
        x = self.fc(x[:, -1, :])
        return self.sigmoid(x)

# 평가 함수 정의
def expected_calibration_error(y_true, y_prob, n_bins=10):
    prob_true, prob_pred = calibration_curve(y_true, y_prob, n_bins=n_bins, strategy='uniform')
    bin_totals = np.histogram(y_prob, bins=np.linspace(0, 1, n_bins + 1), density=False)[0]
    non_empty_bins = bin_totals > 0
    bin_weights = bin_totals / len(y_prob)
    bin_weights = bin_weights[non_empty_bins]
    prob_true = prob_true[:len(bin_weights)]
    prob_pred = prob_pred[:len(bin_weights)]
    ece = np.sum(bin_weights * np.abs(prob_true - prob_pred))
    return ece

def auc_brier_ece(answer_df, submission_df):
    # Check for missing values in submission_df
    if submission_df.isnull().values.any():
        raise ValueError("The submission dataframe contains missing values.")

    # Check if the number and names of columns are the same in both dataframes
    if len(answer_df.columns) != len(submission_df.columns) or not all(answer_df.columns == submission_df.columns):
        raise ValueError("The columns of the answer and submission dataframes do not match.")
        
    submission_df = submission_df[submission_df.index.isin(answer_df.index)]
    submission_df.index = range(submission_df.shape[0])
    
    # Calculate AUC for each class
    auc_scores = []
    for column in answer_df.columns:
        y_true = answer_df[column]
        y_scores = submission_df[column]
        auc = roc_auc_score(y_true, y_scores)
        auc_scores.append(auc)

    # Calculate mean AUC
    mean_auc = np.mean(auc_scores)

    brier_scores = []
    ece_scores = []
    
    # Calculate Brier Score and ECE for each class
    for column in answer_df.columns:
        y_true = answer_df[column].values
        y_prob = submission_df[column].values
        
        # Brier Score
        brier = mean_squared_error(y_true, y_prob)
        brier_scores.append(brier)
        
        # ECE
        ece = expected_calibration_error(y_true, y_prob)
        ece_scores.append(ece)
    
    # Calculate mean Brier Score and mean ECE
    mean_brier = np.mean(brier_scores)
    mean_ece = np.mean(ece_scores)
    
    # Calculate combined score
    combined_score = 0.5 * (1 - mean_auc) + 0.25 * mean_brier + 0.25 * mean_ece
    
    return combined_score

def train(model, optimizer, train_loader, val_loader, device):
    model.to(device)
    criterion = nn.BCELoss().to(device)
    
    best_val_score = 0
    best_model = None
    val_labels = y_val.cpu().numpy()
    
    for epoch in range(1, CONFIG.N_EPOCHS+1):
        model.train()
        train_loss = []
        for features, labels in tqdm(iter(train_loader)):
            features = features.float().to(device)
            labels = labels.float().to(device)
            
            optimizer.zero_grad()
            
            output = model(features)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score, val_outputs = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        
        combined_score = auc_brier_ece(pd.DataFrame(val_labels), pd.DataFrame(val_outputs))
        
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val AUC : [{_val_score:.5f}] Combined Score: [{combined_score:.5f}]')
            
        if best_val_score < _val_score:
            best_val_score = _val_score
            best_model = model
    
    return best_model

def multiLabel_AUC(y_true, y_scores):
    auc_scores = []
    for i in range(y_true.shape[1]):
        auc = roc_auc_score(y_true[:, i], y_scores[:, i])
        auc_scores.append(auc)
    mean_auc_score = np.mean(auc_scores)
    return mean_auc_score
    
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss, all_labels, all_probs = [], [], []
    
    with torch.no_grad():
        for features, labels in tqdm(iter(val_loader)):
            features = features.float().to(device)
            labels = labels.float().to(device)
            
            probs = model(features)
            
            loss = criterion(probs, labels)

            val_loss.append(loss.item())

            all_labels.append(labels.cpu().numpy())
            all_probs.append(probs.cpu().numpy())
        
        _val_loss = np.mean(val_loss)

        all_labels = np.concatenate(all_labels, axis=0)
        all_probs = np.concatenate(all_probs, axis=0)
        
        auc_score = multiLabel_AUC(all_labels, all_probs)
    
    return _val_loss, auc_score, all_probs

# 최적의 하이퍼파라미터로 모델 학습
model = CNNRNN(input_dim=CONFIG.N_FEATURES, hidden_dim=CONFIG.HIDDEN_DIM, output_dim=CONFIG.N_CLASSES, dropout_rate=CONFIG.DROPOUT_RATE)
optimizer = torch.optim.Adam(params=model.parameters(), lr=CONFIG.LR)

infer_model = train(model, optimizer, train_loader, val_loader, device)

# Unlabeled 데이터에 대해 pseudo-labeling
unlabeled_path = './unlabeled_data'
unlabeled_files = [os.path.join(unlabeled_path, f) for f in os.listdir(unlabeled_path) if f.endswith('.ogg')]

unlabeled_features = get_features_and_labels(pd.DataFrame({'path': unlabeled_files}), False)
unlabeled_features = np.array(unlabeled_features)
unlabeled_dataset = CustomDataset(unlabeled_features, torch.zeros((len(unlabeled_features), CONFIG.N_CLASSES)))
unlabeled_loader = DataLoader(unlabeled_dataset, batch_size=CONFIG.BATCH_SIZE, shuffle=False)

def pseudo_labeling(model, loader, device):
    model.to(device)
    model.eval()
    pseudo_labels = []
    with torch.no_grad():
        for features, _ in tqdm(iter(loader)):
            features = features.float().to(device)
            probs = model(features)
            pseudo_labels.append(probs.cpu().detach().numpy())
    return np.concatenate(pseudo_labels, axis=0)

pseudo_labels = pseudo_labeling(infer_model, unlabeled_loader, device)
pseudo_labels = (pseudo_labels > 0.9).astype(int)  # 임계값 0.9을 사용하여 pseudo-label 생성

# Pseudo-labeled 데이터를 학습 데이터에 추가
pseudo_labeled_dataset = CustomDataset(unlabeled_features, torch.tensor(pseudo_labels).float())
train_dataset_combined = torch.utils.data.ConcatDataset([train_dataset, pseudo_labeled_dataset])
train_loader_combined = DataLoader(train_dataset_combined, batch_size=CONFIG.BATCH_SIZE, shuffle=True)

# 모델을 pseudo-labeled 데이터를 포함하여 재학습
infer_model = train(model, optimizer, train_loader_combined, val_loader, device)

# 테스트 데이터 예측
test = pd.read_csv(os.path.join(CONFIG.ROOT_FOLDER, 'test.csv'))
test_features = get_features_and_labels(test, False)
test_features = np.array(test_features)
test_dataset = CustomDataset(test_features, torch.zeros((len(test_features), CONFIG.N_CLASSES)))
test_loader = DataLoader(test_dataset, batch_size=CONFIG.BATCH_SIZE, shuffle=False)

def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    predictions = []
    with torch.no_grad():
        for features, _ in tqdm(iter(test_loader)):
            features = features.float().to(device)
            probs = model(features)
            probs  = probs.cpu().detach().numpy()
            predictions += probs.tolist()
    return predictions

preds = inference(infer_model, test_loader, device)

submit = pd.read_csv(os.path.join(CONFIG.ROOT_FOLDER, 'sample_submission.csv'))
submit.iloc[:, 1:] = preds
submit.head()

combined_score_before_test = auc_brier_ece(pd.DataFrame(y_val.cpu().numpy()), pd.DataFrame(preds))
print(f'Final Combined Score before test: {combined_score_before_test:.5f}')

submit.to_csv('./best_model_submit2.csv', index=False)


Using device: cpu
label ['fake' 'real']


55438it [5:00:14,  3.08it/s]
100%|██████████| 348/348 [00:35<00:00,  9.83it/s]


Epoch [1], Train Loss : [0.27687] Val Loss : [0.10202] Val AUC : [0.99343] Combined Score: [0.01240]


100%|██████████| 348/348 [00:37<00:00,  9.16it/s]


Epoch [2], Train Loss : [0.11203] Val Loss : [0.15230] Val AUC : [0.99541] Combined Score: [0.02864]


...

Epoch [9], Train Loss : [0.01568] Val Loss : [0.03590] Val AUC : [0.99969] Combined Score: [0.00588]


100%|██████████| 348/348 [00:49<00:00,  7.09it/s]


Epoch [10], Train Loss : [0.01657] Val Loss : [0.01351] Val AUC : [0.99983] Combined Score: [0.00153]


50000it [5:38:50,  2.46it/s]
100%|██████████| 1563/1563 [02:44<00:00,  9.48it/s]


Final Combined Score before test: 0.49510
